Request from 1/27/2021

data from 1950 to 2019 as monthly sum in netcdf format for:
- Precipitation (input)
- actual ET
- recharge (deep drainage)
- surface runoff

Precipitation (input) is the only one that needs to be summed up, the output data is already summed up.

outputs are based on "run XX"

code is based on TonyB script "00-Create-Wzell-ET-Outputs-Sums-PPT-Only-its-SPECIAL" and "01-create-netcdfs-and-tarballs-cleanup.ipynb
"

In [1]:
%env CURL_CA_BUNDLE=/etc/ssl/certs/ca-certificates.crt

env: CURL_CA_BUNDLE=/etc/ssl/certs/ca-certificates.crt


In [2]:
import os
from enduserLib.sum_mydata import monthly_sum
from enduserLib.xr_netcdf import xr_build_cube_concat_ds
from enduserLib.tony_stuff import tony_mkdir

In [3]:
def create_s3_list_of_days(main_prefix, year, input_name='PPT_'):
    the_list = []
    for i in range(1,366):
        month = f'{i:03d}'
        file_object = main_prefix +os.sep+ str(year) +os.sep+ input_name + str(year) + month + '.tif'
        the_list.append(file_object)
    return the_list

In [4]:
# to run just 1 year

#main_prefix = 's3://dev-et-data/in/DelawareRiverBasin/PPT'
#year = 2003
#output_name = 'PPT_'
#ppt_list = create_s3_list_of_days(main_prefix, year, output_name)

In [5]:
def _mkdir(directory):
    try:
        os.stat(directory)
    except:
        os.mkdir(directory)

# Create PPT Sum Tifs

In [6]:
#_mkdir('./junkboxpptsum')

In [7]:
main_prefix = 's3://dev-et-data/in/DelawareRiverBasin/PPT'
out_dir = './junkboxpptsum'
input_name = 'PPT_'
out_product = 'ppt_'


# 2000-2018 was already created for a previous request
# just do 1950 - 1999
# monthly function comes from http://10.12.68.246/user/wzell/tree/opt/enduser/enduserLib/enduserLib
print('Data is outputted to: {}'.format(out_dir))
#for year in range(2019,2020):
#    file_list = create_s3_list_of_days(main_prefix, year, input_name)
#    monthly_sum(file_list, out_dir, out_product, year)

Data is outputted to: ./junkboxpptsum


In [8]:
! ls -lh ./junkboxpptsum

total 30M
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:18 ppt_195001.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:18 ppt_195002.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:18 ppt_195003.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:19 ppt_195004.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:19 ppt_195005.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:19 ppt_195006.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:19 ppt_195007.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:19 ppt_195008.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:19 ppt_195009.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:19 ppt_195010.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:19 ppt_195011.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:19 ppt_195012.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 35K Jan 28 19:19 ppt_195101.tif
-rw-r--r-- 1 jupyter-wzell jupyter-wzell 

In [9]:
! date

Fri Jan 29 15:32:20 UTC 2021


# Create netcdfs as output format

In [10]:
tony_mkdir('./ncdir')

In [11]:
def create_s3_list_of_months(main_prefix, year, output_name='etasw_'):
    the_list = []
    for i in range(1,13):
        month = f'{i:02d}'
        file_object = main_prefix  + str(year) +os.sep+ output_name + str(year) + month + '.tif'
        the_list.append(file_object)
    return the_list

In [12]:
def netcdf_creator(output_name):
    #working_bucket = 'dev-et-data'
    main_prefix = 's3://dev-et-data/enduser/DelawareRiverBasin/Run09_13_2020/ward_sandford_customer/'
    years = range(1950,2020)
    #output_name = 'etasw_'
    #output_name = 'dd_'
    #output_name = 'srf_'

    for year in years:
        my_tifs = create_s3_list_of_months(main_prefix, year, output_name)
        ds = xr_build_cube_concat_ds(my_tifs, output_name)
        nc_file_name = './ncdir' +os.sep+ output_name + str(year) + '.nc'
        ds.to_netcdf(nc_file_name, engine='h5netcdf')

In [13]:
# outs = ['etasw_', 'dd_', 'srf_']

In [14]:
# for out in outs:
#     netcdf_creator(out)

In [15]:
# ! ls ./ncdir

# PPT just wants to be different

In [16]:
def create_local_list_of_months(main_prefix, year, output_name='ppt_'):
    the_list = []
    for i in range(1,13):
        month = f'{i:02d}'
        #file_object = main_prefix  + str(year) + '/' + output_name + str(year) + month + '.tif'
        file_object = main_prefix + output_name + str(year) + month + '.tif'
        the_list.append(file_object)
    return the_list

In [17]:
main_prefix = './junkboxpptsum/'
years = range(1950,2020)
output_name='ppt_'
for year in years:
        my_tifs = create_local_list_of_months(main_prefix, year, output_name)
        ds = xr_build_cube_concat_ds(my_tifs, output_name)
        nc_file_name = './ncdir' +os.sep+ output_name + str(year) + '.nc'
        ds.to_netcdf(nc_file_name, engine='h5netcdf')

./junkboxpptsum/ppt_195001.tif
./junkboxpptsum/ppt_195001.tif 0.01099395751953125
195001
./junkboxpptsum/ppt_195002.tif
./junkboxpptsum/ppt_195002.tif 0.016272783279418945
195002
./junkboxpptsum/ppt_195003.tif
./junkboxpptsum/ppt_195003.tif 0.021186113357543945
195003
./junkboxpptsum/ppt_195004.tif
./junkboxpptsum/ppt_195004.tif 0.02444291114807129
195004
./junkboxpptsum/ppt_195005.tif
./junkboxpptsum/ppt_195005.tif 0.027622699737548828
195005
./junkboxpptsum/ppt_195006.tif
./junkboxpptsum/ppt_195006.tif 0.030716657638549805
195006
./junkboxpptsum/ppt_195007.tif
./junkboxpptsum/ppt_195007.tif 0.034223318099975586
195007
./junkboxpptsum/ppt_195008.tif
./junkboxpptsum/ppt_195008.tif 0.03705406188964844
195008
./junkboxpptsum/ppt_195009.tif
./junkboxpptsum/ppt_195009.tif 0.03999590873718262
195009
./junkboxpptsum/ppt_195010.tif
./junkboxpptsum/ppt_195010.tif 0.04299211502075195
195010
./junkboxpptsum/ppt_195011.tif
./junkboxpptsum/ppt_195011.tif 0.045693159103393555
195011
./junkboxpptsum

./junkboxpptsum/ppt_195801.tif
./junkboxpptsum/ppt_195801.tif 0.004498481750488281
195801
./junkboxpptsum/ppt_195802.tif
./junkboxpptsum/ppt_195802.tif 0.007093667984008789
195802
./junkboxpptsum/ppt_195803.tif
./junkboxpptsum/ppt_195803.tif 0.009616851806640625
195803
./junkboxpptsum/ppt_195804.tif
./junkboxpptsum/ppt_195804.tif 0.012821435928344727
195804
./junkboxpptsum/ppt_195805.tif
./junkboxpptsum/ppt_195805.tif 0.01523733139038086
195805
./junkboxpptsum/ppt_195806.tif
./junkboxpptsum/ppt_195806.tif 0.01771402359008789
195806
./junkboxpptsum/ppt_195807.tif
./junkboxpptsum/ppt_195807.tif 0.020297765731811523
195807
./junkboxpptsum/ppt_195808.tif
./junkboxpptsum/ppt_195808.tif 0.023363113403320312
195808
./junkboxpptsum/ppt_195809.tif
./junkboxpptsum/ppt_195809.tif 0.02555537223815918
195809
./junkboxpptsum/ppt_195810.tif
./junkboxpptsum/ppt_195810.tif 0.028459548950195312
195810
./junkboxpptsum/ppt_195811.tif
./junkboxpptsum/ppt_195811.tif 0.030945301055908203
195811
./junkboxppts

./junkboxpptsum/ppt_196601.tif 0.0032405853271484375
196601
./junkboxpptsum/ppt_196602.tif
./junkboxpptsum/ppt_196602.tif 0.006366729736328125
196602
./junkboxpptsum/ppt_196603.tif
./junkboxpptsum/ppt_196603.tif 0.009535074234008789
196603
./junkboxpptsum/ppt_196604.tif
./junkboxpptsum/ppt_196604.tif 0.012769460678100586
196604
./junkboxpptsum/ppt_196605.tif
./junkboxpptsum/ppt_196605.tif 0.015825986862182617
196605
./junkboxpptsum/ppt_196606.tif
./junkboxpptsum/ppt_196606.tif 0.0191347599029541
196606
./junkboxpptsum/ppt_196607.tif
./junkboxpptsum/ppt_196607.tif 0.022734880447387695
196607
./junkboxpptsum/ppt_196608.tif
./junkboxpptsum/ppt_196608.tif 0.026410818099975586
196608
./junkboxpptsum/ppt_196609.tif
./junkboxpptsum/ppt_196609.tif 0.02950739860534668
196609
./junkboxpptsum/ppt_196610.tif
./junkboxpptsum/ppt_196610.tif 0.03191089630126953
196610
./junkboxpptsum/ppt_196611.tif
./junkboxpptsum/ppt_196611.tif 0.03448843955993652
196611
./junkboxpptsum/ppt_196612.tif
./junkboxpptsu

./junkboxpptsum/ppt_197501.tif
./junkboxpptsum/ppt_197501.tif 0.004291057586669922
197501
./junkboxpptsum/ppt_197502.tif
./junkboxpptsum/ppt_197502.tif 0.006664276123046875
197502
./junkboxpptsum/ppt_197503.tif
./junkboxpptsum/ppt_197503.tif 0.009940385818481445
197503
./junkboxpptsum/ppt_197504.tif
./junkboxpptsum/ppt_197504.tif 0.01226663589477539
197504
./junkboxpptsum/ppt_197505.tif
./junkboxpptsum/ppt_197505.tif 0.015586137771606445
197505
./junkboxpptsum/ppt_197506.tif
./junkboxpptsum/ppt_197506.tif 0.021587848663330078
197506
./junkboxpptsum/ppt_197507.tif
./junkboxpptsum/ppt_197507.tif 0.02445816993713379
197507
./junkboxpptsum/ppt_197508.tif
./junkboxpptsum/ppt_197508.tif 0.027904510498046875
197508
./junkboxpptsum/ppt_197509.tif
./junkboxpptsum/ppt_197509.tif 0.030258893966674805
197509
./junkboxpptsum/ppt_197510.tif
./junkboxpptsum/ppt_197510.tif 0.03366994857788086
197510
./junkboxpptsum/ppt_197511.tif
./junkboxpptsum/ppt_197511.tif 0.03717041015625
197511
./junkboxpptsum/p

./junkboxpptsum/ppt_198401.tif 0.0039005279541015625
198401
./junkboxpptsum/ppt_198402.tif
./junkboxpptsum/ppt_198402.tif 0.0064411163330078125
198402
./junkboxpptsum/ppt_198403.tif
./junkboxpptsum/ppt_198403.tif 0.009121417999267578
198403
./junkboxpptsum/ppt_198404.tif
./junkboxpptsum/ppt_198404.tif 0.01170039176940918
198404
./junkboxpptsum/ppt_198405.tif
./junkboxpptsum/ppt_198405.tif 0.016877412796020508
198405
./junkboxpptsum/ppt_198406.tif
./junkboxpptsum/ppt_198406.tif 0.021399259567260742
198406
./junkboxpptsum/ppt_198407.tif
./junkboxpptsum/ppt_198407.tif 0.024166584014892578
198407
./junkboxpptsum/ppt_198408.tif
./junkboxpptsum/ppt_198408.tif 0.027824878692626953
198408
./junkboxpptsum/ppt_198409.tif
./junkboxpptsum/ppt_198409.tif 0.03020954132080078
198409
./junkboxpptsum/ppt_198410.tif
./junkboxpptsum/ppt_198410.tif 0.03264045715332031
198410
./junkboxpptsum/ppt_198411.tif
./junkboxpptsum/ppt_198411.tif 0.03549051284790039
198411
./junkboxpptsum/ppt_198412.tif
./junkboxppt

./junkboxpptsum/ppt_199301.tif
./junkboxpptsum/ppt_199301.tif 0.0038614273071289062
199301
./junkboxpptsum/ppt_199302.tif
./junkboxpptsum/ppt_199302.tif 0.007504701614379883
199302
./junkboxpptsum/ppt_199303.tif
./junkboxpptsum/ppt_199303.tif 0.01109623908996582
199303
./junkboxpptsum/ppt_199304.tif
./junkboxpptsum/ppt_199304.tif 0.014366388320922852
199304
./junkboxpptsum/ppt_199305.tif
./junkboxpptsum/ppt_199305.tif 0.01721930503845215
199305
./junkboxpptsum/ppt_199306.tif
./junkboxpptsum/ppt_199306.tif 0.019632577896118164
199306
./junkboxpptsum/ppt_199307.tif
./junkboxpptsum/ppt_199307.tif 0.02242445945739746
199307
./junkboxpptsum/ppt_199308.tif
./junkboxpptsum/ppt_199308.tif 0.02446889877319336
199308
./junkboxpptsum/ppt_199309.tif
./junkboxpptsum/ppt_199309.tif 0.027015209197998047
199309
./junkboxpptsum/ppt_199310.tif
./junkboxpptsum/ppt_199310.tif 0.030717134475708008
199310
./junkboxpptsum/ppt_199311.tif
./junkboxpptsum/ppt_199311.tif 0.03407716751098633
199311
./junkboxpptsu

./junkboxpptsum/ppt_200202.tif 0.009059906005859375
200202
./junkboxpptsum/ppt_200203.tif
./junkboxpptsum/ppt_200203.tif 0.012956380844116211
200203
./junkboxpptsum/ppt_200204.tif
./junkboxpptsum/ppt_200204.tif 0.016106367111206055
200204
./junkboxpptsum/ppt_200205.tif
./junkboxpptsum/ppt_200205.tif 0.018914222717285156
200205
./junkboxpptsum/ppt_200206.tif
./junkboxpptsum/ppt_200206.tif 0.021317481994628906
200206
./junkboxpptsum/ppt_200207.tif
./junkboxpptsum/ppt_200207.tif 0.024588823318481445
200207
./junkboxpptsum/ppt_200208.tif
./junkboxpptsum/ppt_200208.tif 0.026930570602416992
200208
./junkboxpptsum/ppt_200209.tif
./junkboxpptsum/ppt_200209.tif 0.029209375381469727
200209
./junkboxpptsum/ppt_200210.tif
./junkboxpptsum/ppt_200210.tif 0.03488039970397949
200210
./junkboxpptsum/ppt_200211.tif
./junkboxpptsum/ppt_200211.tif 0.03824210166931152
200211
./junkboxpptsum/ppt_200212.tif
./junkboxpptsum/ppt_200212.tif 0.041503190994262695
200212
./junkboxpptsum/ppt_200301.tif
./junkboxppt

./junkboxpptsum/ppt_201101.tif 0.0032973289489746094
201101
./junkboxpptsum/ppt_201102.tif
./junkboxpptsum/ppt_201102.tif 0.0059697628021240234
201102
./junkboxpptsum/ppt_201103.tif
./junkboxpptsum/ppt_201103.tif 0.010361909866333008
201103
./junkboxpptsum/ppt_201104.tif
./junkboxpptsum/ppt_201104.tif 0.015291929244995117
201104
./junkboxpptsum/ppt_201105.tif
./junkboxpptsum/ppt_201105.tif 0.019810914993286133
201105
./junkboxpptsum/ppt_201106.tif
./junkboxpptsum/ppt_201106.tif 0.023265600204467773
201106
./junkboxpptsum/ppt_201107.tif
./junkboxpptsum/ppt_201107.tif 0.02670145034790039
201107
./junkboxpptsum/ppt_201108.tif
./junkboxpptsum/ppt_201108.tif 0.029603958129882812
201108
./junkboxpptsum/ppt_201109.tif
./junkboxpptsum/ppt_201109.tif 0.032497406005859375
201109
./junkboxpptsum/ppt_201110.tif
./junkboxpptsum/ppt_201110.tif 0.03557419776916504
201110
./junkboxpptsum/ppt_201111.tif
./junkboxpptsum/ppt_201111.tif 0.03843808174133301
201111
./junkboxpptsum/ppt_201112.tif
./junkboxpp

In [18]:
! ls ./ncdir

ppt_1950.nc  ppt_1962.nc  ppt_1974.nc  ppt_1986.nc  ppt_1998.nc  ppt_2010.nc
ppt_1951.nc  ppt_1963.nc  ppt_1975.nc  ppt_1987.nc  ppt_1999.nc  ppt_2011.nc
ppt_1952.nc  ppt_1964.nc  ppt_1976.nc  ppt_1988.nc  ppt_2000.nc  ppt_2012.nc
ppt_1953.nc  ppt_1965.nc  ppt_1977.nc  ppt_1989.nc  ppt_2001.nc  ppt_2013.nc
ppt_1954.nc  ppt_1966.nc  ppt_1978.nc  ppt_1990.nc  ppt_2002.nc  ppt_2014.nc
ppt_1955.nc  ppt_1967.nc  ppt_1979.nc  ppt_1991.nc  ppt_2003.nc  ppt_2015.nc
ppt_1956.nc  ppt_1968.nc  ppt_1980.nc  ppt_1992.nc  ppt_2004.nc  ppt_2016.nc
ppt_1957.nc  ppt_1969.nc  ppt_1981.nc  ppt_1993.nc  ppt_2005.nc  ppt_2017.nc
ppt_1958.nc  ppt_1970.nc  ppt_1982.nc  ppt_1994.nc  ppt_2006.nc  ppt_2018.nc
ppt_1959.nc  ppt_1971.nc  ppt_1983.nc  ppt_1995.nc  ppt_2007.nc  ppt_2019.nc
ppt_1960.nc  ppt_1972.nc  ppt_1984.nc  ppt_1996.nc  ppt_2008.nc
ppt_1961.nc  ppt_1973.nc  ppt_1985.nc  ppt_1997.nc  ppt_2009.nc


In [19]:
tony_mkdir('./tarballs')

In [20]:
#! tar cvfz ./tarballs/wzell_etasw.tgz ./ncdir/eta*.nc
#! tar cvfz ./tarballs/wzell_dd.tgz ./ncdir/dd*.nc
#! tar cvfz ./tarballs/wzell_srf.tgz ./ncdir/srf*.nc
! tar cvfz ./tarballs/wzell_ppt_sum.tgz ./ncdir/ppt_*.nc

./ncdir/ppt_1950.nc
./ncdir/ppt_1951.nc
./ncdir/ppt_1952.nc
./ncdir/ppt_1953.nc
./ncdir/ppt_1954.nc
./ncdir/ppt_1955.nc
./ncdir/ppt_1956.nc
./ncdir/ppt_1957.nc
./ncdir/ppt_1958.nc
./ncdir/ppt_1959.nc
./ncdir/ppt_1960.nc
./ncdir/ppt_1961.nc
./ncdir/ppt_1962.nc
./ncdir/ppt_1963.nc
./ncdir/ppt_1964.nc
./ncdir/ppt_1965.nc
./ncdir/ppt_1966.nc
./ncdir/ppt_1967.nc
./ncdir/ppt_1968.nc
./ncdir/ppt_1969.nc
./ncdir/ppt_1970.nc
./ncdir/ppt_1971.nc
./ncdir/ppt_1972.nc
./ncdir/ppt_1973.nc
./ncdir/ppt_1974.nc
./ncdir/ppt_1975.nc
./ncdir/ppt_1976.nc
./ncdir/ppt_1977.nc
./ncdir/ppt_1978.nc
./ncdir/ppt_1979.nc
./ncdir/ppt_1980.nc
./ncdir/ppt_1981.nc
./ncdir/ppt_1982.nc
./ncdir/ppt_1983.nc
./ncdir/ppt_1984.nc
./ncdir/ppt_1985.nc
./ncdir/ppt_1986.nc
./ncdir/ppt_1987.nc
./ncdir/ppt_1988.nc
./ncdir/ppt_1989.nc
./ncdir/ppt_1990.nc
./ncdir/ppt_1991.nc
./ncdir/ppt_1992.nc
./ncdir/ppt_1993.nc
./ncdir/ppt_1994.nc
./ncdir/ppt_1995.nc
./ncdir/ppt_1996.nc
./ncdir/ppt_1997.nc
./ncdir/ppt_1998.nc
./ncdir/ppt_1999.nc


In [21]:
! ls ./tarballs

wzell_ppt_sum.tgz


In [22]:
# copy tar files from tarballs to current dir ()aware
! echo scp -r jupyter-wzell@10.12.68.246:`pwd`/tarballs . 

scp -r jupyter-wzell@10.12.68.246:/home/jupyter-wzell/opt/enduser/00-wzell-2-0/tarballs .
